In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Concatenate, Embedding, Reshape, Dot
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 1: Load and preprocess the MovieLens dataset
file_path = '/content/drive/MyDrive/Colab Notebooks/FINAL Project/data/ml-latest'
movies = pd.read_csv(f'{file_path}/movies.csv')
movies['genres'] = movies['genres'].str.split('|')
print(movies.shape)
movies.head()

(58098, 3)


,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [ ]:
ratings = pd.read_csv(f'{file_path}/ratings.csv')
ratings = ratings.drop('timestamp',axis=1)
print(ratings.shape)
ratings.head()

(27753444, 3)


,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [ ]:
# Take a smaller sample of the user ratings
sample_size = 400000
ratings = ratings[0:sample_size].copy()
print(ratings.shape)
ratings.head()

(400000, 3)


,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5
...,...,...,...
399995,4084,541,4.0
399996,4084,552,2.0
399997,4084,586,2.0
399998,4084,587,2.0


In [ ]:
tags = pd.read_csv(f'{file_path}/tags_filtered_120.csv')
print(tags.shape)
tags.head()

(621449, 4)


,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,260,sci-fi,1442169410
2,14,260,space action,1442169421
3,14,318,imdb top 250,1442615195
4,14,318,justice,1442615192


In [ ]:
#make sure all dataframes have the same movies included
movies = movies[movies['movieId'].isin(ratings['movieId'])]
movies = movies.reset_index(drop=True)

tags = tags[tags['movieId'].isin(ratings['movieId'])]
tags = tags.reset_index(drop=True)

movies = movies[movies['movieId'].isin(tags['movieId'])]
movies = movies.reset_index(drop=True)

ratings = ratings[ratings['movieId'].isin(tags['movieId'])]
ratings = ratings.reset_index(drop=True)

In [ ]:
print(f"There are: {ratings['userId'].nunique()} unqiue users")
print(f"There are: {ratings['movieId'].nunique()} unqiue films in the ratings dataframe")
print(f"There are: {movies['movieId'].nunique()} unqiue films in the movies dataframe")
print(f"There are: {tags['movieId'].nunique()} unqiue films in the tags dataframe")
print(f"There are: {tags['tag'].nunique()} unqiue tags in the tags dataframe")

There are: 4084 unqiue users
There are: 14864 unqiue films in the ratings dataframe
There are: 14864 unqiue films in the movies dataframe
There are: 14864 unqiue films in the tags dataframe
There are: 1282 unqiue tags in the tags dataframe


In [ ]:
# #remove tags that occur less than 120 times from the tags dataframe
# from collections import Counter

# #get values for every tag and count the occurances
# all_tags = tags['tag'].values
# tag_counts = Counter(all_tags)

# # Sort the tags by their frequencies in descending order
# sorted_tags = sorted(tag_counts.items(), key=lambda x: x[1], reverse=True)

# # Print the sorted tags(its a very long output)
# # for tag, count in sorted_tags:
# #     print(tag, count)

# # Create a list of tags to remove (occurring less than 120 times)
# no_of_tag_occurances = 120
# tags_to_remove = [tag for tag, count in tag_counts.items() if count < no_of_tag_occurances]
# print(len(tags_to_remove), "tags to be removed from dataframe")

# #iterate through tags dataframe and find the index numbers of rows to be removed
# index_to_remove = []
# for num, tag in enumerate(tags['tag']):
#     if tag in tags_to_remove:
#         index_to_remove.append(num)

# tags_filtered = tags.drop(index_to_remove)
# tags_filtered = tags_filtered.reset_index(drop=True)
# print(tags_filtered.shape)
# tags_filtered.head()

In [ ]:
# tags_filtered.to_csv(f'{file_path}/tags_filtered_120.csv', index=False)

# Join Data

In [ ]:
#aggregate tags into list by movie id and attach to dataframe
tags_agg = tags.groupby('movieId')['tag'].agg(list)
movies_tag_df = pd.merge(movies, tags_agg, how='inner', on='movieId')

#eliminate duplicate tag in tag lists by turning into a set and then back into a list
movies_tag_df['tag'] = movies_tag_df['tag'].apply(lambda tags: list({tag for tag in tags}))

#display dataframe
print(movies_tag_df.shape)
movies_tag_df.head()

(14864, 4)


,movieId,title,genres,tag
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[friends, kids, sci-fi, erlend's DVDs, classic..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[time travel, CLV, kids, comedy, children, ani..."
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[CLV, comedy, sequel, good soundtrack, duringc..."
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]","[CLV, characters, chick flick, divorce, based ..."
4,5,Father of the Bride Part II (1995),[Comedy],"[CLV, parent child relationship, sequel, Fanta..."


In [ ]:
# Extract the genre information
genres = movies_tag_df['genres'].values

# Perform one-hot encoding on the genre lists
mlb = MultiLabelBinarizer()
genre_features = mlb.fit_transform(genres)

# Retrieve the list of genre classes
genre_classes = mlb.classes_

# Create a new dataframe with the one-hot encoded genre features
genre_df = pd.DataFrame(genre_features, columns=genre_classes)
genre_df.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Extract the tag information
tags = movies_tag_df['tag'].values

# Perform one-hot encoding on the tag lists
mlb = MultiLabelBinarizer()
tag_features = mlb.fit_transform(tags)

# Retrieve the list of tag classes
tag_classes = mlb.classes_

# Create a new dataframe with the one-hot encoded tag features
tag_df = pd.DataFrame(tag_features, columns=tag_classes)
tag_df.head()

,007,01/11,02/11,03/11,05/11,06/11,11/10,11/11,18th century,1920s,...,workplace,world war ii,wormhole,writer,writers,writing,wry,youth,zombie,zombies
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
movies_features = pd.concat([movies_tag_df,genre_df,tag_df],axis=1)
print(movies_features.shape)
movies_features.head()

(14864, 1306)


,movieId,title,genres,tag,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,workplace,world war ii,wormhole,writer,writers,writing,wry,youth,zombie,zombies
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[friends, kids, sci-fi, erlend's DVDs, classic...",0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[time travel, CLV, kids, comedy, children, ani...",0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[CLV, comedy, sequel, good soundtrack, duringc...",0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]","[CLV, characters, chick flick, divorce, based ...",0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),[Comedy],"[CLV, parent child relationship, sequel, Fanta...",0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
movielens_df = pd.merge(ratings,movies_features,how='left', on = 'movieId')
movielens_df = movielens_df.drop(['title','genres','tag'],axis=1)
print(movielens_df.shape)
movielens_df.head()

(397432, 1305)


,userId,movieId,rating,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,workplace,world war ii,wormhole,writer,writers,writing,wry,youth,zombie,zombies
0,1,307,3.5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,481,3.5,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
2,1,1091,1.5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1257,4.5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1449,4.5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Normalize the ratings between 0 and 1
max_rating = movielens_df['rating'].max()
min_rating = movielens_df['rating'].min()
print(max_rating)
print(min_rating)

movielens_df['rating'] = (movielens_df['rating'] - min_rating) / (max_rating - min_rating)
movielens_df.head()

5.0
0.5


,userId,movieId,rating,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,workplace,world war ii,wormhole,writer,writers,writing,wry,youth,zombie,zombies
0,1,307,0.666667,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,481,0.666667,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
2,1,1091,0.222222,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1257,0.888889,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1449,0.888889,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Split the dataset into training and testing sets
train_data, test_data = train_test_split(movielens_df, test_size=0.2, random_state=42)

# Determine the maximum values of userId and movieId for embedding layer sizes
max_user_id = ratings['userId'].max()
max_movie_id = ratings['movieId'].max()

# Determine the number of genre and tag features
genre_len = len(genre_df.columns)
tag_len = len(tag_df.columns)

# Prepare the input data for the neural network
train_user_ids = train_data['userId'].values
train_movie_ids = train_data['movieId'].values
train_genres = train_data.iloc[:, 3:23].values
train_tags = train_data.iloc[:, 23:].values
train_ratings = train_data['rating'].values

test_user_ids = test_data['userId'].values
test_movie_ids = test_data['movieId'].values
test_genres = test_data.iloc[:, 3:23].values
test_tags = test_data.iloc[:, 23:].values
test_ratings = test_data['rating'].values

In [ ]:
print(train_user_ids.shape)
print(train_movie_ids.shape)
print(train_genres.shape)
print(train_tags.shape)
print(train_ratings.shape)

(317945,)
(317945,)
(317945, 20)
(317945, 1282)
(317945,)


In [ ]:
# Define the embedding sizes for users and movies
embedding_size = 50

############# COLLABORATIVE FILTERING ###############################################################################################
# Input Layers
user_input = Input(name='user_input', shape=(1,))
movie_input = Input(name='movie_input', shape=(1,))


# Matrix Factorization
## embed & reshape
cf_user_embedding = Embedding(name='cf_user_embedding', input_dim = max_user_id+1, output_dim = embedding_size)(user_input)
cf_user_reshape = Reshape(name = 'cf_user_reshape', target_shape=(embedding_size,))(cf_user_embedding)
cf_movie_embedding = Embedding(name='cf_movie_embedding', input_dim = max_movie_id+1, output_dim = embedding_size)(movie_input)
cf_movie_reshape = Reshape(name = 'cf_movie_reshape', target_shape=(embedding_size,))(cf_movie_embedding)

## Dot Product of users and movie embeddings
cf_user_movie_dot = Dot(name = 'cf_user_movie_dot', normalize=True, axes=1)([cf_user_reshape, cf_movie_reshape])

#Neural Network
## embed & reshape
nn_user_embedding = Embedding(name='nn_user_embedding', input_dim = max_user_id+1, output_dim = embedding_size)(user_input)
nn_user_reshape = Reshape(name = 'nn_user_reshape', target_shape=(embedding_size,))(nn_user_embedding)
nn_movie_embedding = Embedding(name='nn_movie_embedding', input_dim = max_movie_id+1, output_dim = embedding_size)(movie_input)
nn_movie_reshape = Reshape(name = 'nn_movie_reshape', target_shape=(embedding_size,))(nn_movie_embedding)

## concat & dense
nn_concat = Concatenate()([nn_user_reshape, nn_movie_reshape])
nn_dense = Dense(name="nn_dense", units=int(embedding_size/2), activation='relu')(nn_concat)



############# CONTENT BASED ###############################################################################################

#Genre Features
genre_input = Input(name="genre_input", shape=(genre_len,))
genre_features = Dense(name="genre_features", units=genre_len, activation='relu')(genre_input)
genre_input = Input(name="genre_input", shape=(genre_len,))
genre_features = Dense(name="genre_features", units=genre_len, activation='relu')(genre_input)



#TAG Features
tag_input = Input(name="tag_input", shape=(tag_len,))
tag_features = Dense(name="tag_features", units=tag_len, activation='relu')(tag_input)


############# OUTPUT ######################################################################################################


# Merge all
pred_ratings = Concatenate()([cf_user_movie_dot, nn_dense, genre_features, tag_features])
pred_ratings = Dense(name="pred_ratings", units=1, activation='linear')(pred_ratings)

# Compile
hybrid_model = Model(inputs=[user_input, movie_input,genre_input,tag_input], outputs=pred_ratings, name="hybrid_model")
hybrid_model.compile(loss='mse', optimizer='adam')

In [ ]:
epochs = 10
batch_size = 128

# Train the model
hybrid_model.fit([train_user_ids, train_movie_ids, train_genres, train_tags], train_ratings, epochs = epochs, batch_size=128, shuffle=True, validation_split=0.3)

Epoch 1/10
1739/1739 [==============================] - 449s 257ms/step - loss: 0.0461 - val_loss: 0.0405
Epoch 2/10
1739/1739 [==============================] - 434s 249ms/step - loss: 0.0359 - val_loss: 0.0401
Epoch 3/10
1739/1739 [==============================] - 432s 248ms/step - loss: 0.0213 - val_loss: 0.0428
Epoch 4/10
1739/1739 [==============================] - 438s 252ms/step - loss: 0.0143 - val_loss: 0.0456
Epoch 5/10
1739/1739 [==============================] - 438s 252ms/step - loss: 0.0116 - val_loss: 0.0470
Epoch 6/10
1739/1739 [==============================] - 437s 251ms/step - loss: 0.0099 - val_loss: 0.0483
Epoch 7/10
1739/1739 [==============================] - 436s 250ms/step - loss: 0.0085 - val_loss: 0.0494
Epoch 8/10
1739/1739 [==============================] - 440s 253ms/step - loss: 0.0075 - val_loss: 0.0507
Epoch 9/10
1739/1739 [==============================] - 438s 252ms/step - loss: 0.0068 - val_loss: 0.0515
Epoch 10/10
1739/1739 [=======================

In [ ]:
predictions = hybrid_model.predict([test_user_ids, test_movie_ids, test_genres, test_tags])
print("the highest rating the model predicted is:", np.max(predictions))
rmse = np.sqrt(mean_squared_error(test_ratings, predictions))
print('RMSE:', rmse)

2484/2484 [==============================] - 11s 4ms/step
the highest rating the model predicted is: 1.3801018
RMSE: 0.2288752114494254


In [ ]:
predictions = np.clip(predictions, a_min=0, a_max=1.0)
print("the highest rating the model predicted is:", np.max(predictions))
rmse = np.sqrt(mean_squared_error(test_ratings, predictions))
print('RMSE:', rmse)

the highest rating the model predicted is: 1.0
RMSE: 0.22681131162457566


In [ ]:
#get all the values of the movie Ids
movie_ids = movies_features['movieId'].values
movie_genres = genre_df.values
movie_tags = tag_df.values

# Choose the user for whom you want to make recommendations
test_user = 16

# Get predictions for all movies for the selected user
user_predictions = hybrid_model.predict([np.array([test_user] * len(movie_ids)), movie_ids, movie_genres, movie_tags])

# Unnormalize the ratings
user_predictions = user_predictions * (max_rating - min_rating) + min_rating

# Sort the predictions in descending order
# Exclude the movies that the user has already seen from the recommendations
top_movie_indices = user_predictions.argsort(axis=0)[::-1][:3]

#Clip ratings so ratings above 5.0 are 5 star ratings
user_predictions = np.clip(user_predictions, a_min=None, a_max=5.0)

# Retrieve the movie IDs of the top 3 movies
print("Top 3 Recommended Films for User", test_user)
for index in top_movie_indices:
    movie_title = movies.loc[index, 'title'].values
    rating = user_predictions[index]
    print("Movie:", movie_title[0], "| Predicted Rating:", rating[0][0])

465/465 [==============================] - 2s 4ms/step
Top 3 Recommended Films for User 16
Movie: Enigma (2001) | Predicted Rating: 5.0
Movie: Volver (2006) | Predicted Rating: 5.0
Movie: Wallace & Gromit: The Best of Aardman Animation (1996) | Predicted Rating: 5.0


In [ ]:
# Choose the user for whom you want to make recommendations
test_user = 16

#create user_ratings dataframe
user_ratings = movielens_df[movielens_df['userId'] == test_user][['userId', 'movieId', 'rating']]
user_ratings['rating'] = user_ratings['rating'] * (max_rating - min_rating) + min_rating
user_ratings = pd.merge(user_ratings, movies,on = 'movieId', how='left')
user_films_viewed = user_ratings['movieId'].values

# Create an array of the user ID to match the shape of user_films_viewed
user_id_array = np.array([test_user] * len(user_films_viewed))

#create a array of all the genres & tags from the movies that the user watched
user_movie_features = movies_features[movies_features['movieId'].isin(user_ratings['movieId'])]
user_movie_genres = user_movie_features.iloc[:, 4:24].values
user_movie_tags = user_movie_features.iloc[:, 24:].values


# Get the model predictions for the user and movie IDs
user_predictions = hybrid_model.predict([user_id_array, user_films_viewed, user_movie_genres, user_movie_tags])
user_predictions = user_predictions * (max_rating - min_rating) + min_rating
user_predictions = np.clip(user_predictions, a_min=0, a_max=5.0)

#add a column for predicted rating to the data frame
user_ratings['predicted_rating'] = user_predictions
user_ratings.sort_values(by='rating',ascending=False)

2/2 [==============================] - 0s 14ms/step


,userId,movieId,rating,title,genres,predicted_rating
20,16,252,5.0,I.Q. (1994),"[Comedy, Romance]",4.891121
11,16,215,5.0,Before Sunrise (1995),"[Drama, Romance]",4.061896
36,16,477,5.0,What's Love Got to Do with It? (1993),"[Drama, Musical]",3.457474
31,16,348,5.0,Bullets Over Broadway (1994),[Comedy],5.000000
29,16,339,5.0,While You Were Sleeping (1995),"[Comedy, Romance]",5.000000
26,16,300,5.0,Quiz Show (1994),[Drama],4.674888
25,16,296,5.0,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]",4.507549
22,16,272,5.0,"Madness of King George, The (1994)","[Comedy, Drama]",5.000000
21,16,265,5.0,Like Water for Chocolate (Como agua para choco...,"[Drama, Fantasy, Romance]",5.000000
16,16,232,5.0,Eat Drink Man Woman (Yin shi nan nu) (1994),"[Comedy, Drama, Romance]",4.926651
